<a href="https://colab.research.google.com/github/mugalan/llama-index/blob/main/LLM_Indexing_Storage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Dependencies

In [12]:
import networkx as nx
import json
import pandas as pd
import numpy as np
import time
from datetime import datetime
from pandas import to_datetime
#import plotly.figure_factory as ff
import plotly.graph_objects as go
import json
import plotly.express as px
import requests
import copy
import urllib
import pathlib
import os
from IPython.display import HTML, display
from IPython.core.display import HTML

In [ ]:
!pip install llama-index-core #llama-parse llama-index-readers-file

In [ ]:
!pip install llama-index-llms-openai llama-index-embeddings-openai

In [4]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, Document, StorageContext, ServiceContext, load_index_from_storage
import llama_index.core.settings as settings
import nest_asyncio
nest_asyncio.apply()
OPENAI_API_KEY="api-key"
openai_llm = OpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)
embed_model = OpenAIEmbedding(model="text-embedding-ada-002", api_key=OPENAI_API_KEY)

#Orise-Webservice Client

In [5]:
def get_token_omdAPI(tokenurl,credentials):
    data = {
        "username": credentials['username'],
        "password": credentials['password']
    }
    token = requests.post(tokenurl, json=data)
    return token

def call_omdAPI(webserviceurl,token,apimethod,apidata):
    postdata={"method":apimethod,"data":apidata}
    headers = {"Authorization": f"Token {token}"}
    response = requests.post(webserviceurl, json=postdata, headers=headers)
    return response

##URL Specifications

In [7]:
domain="coco-green.orise.lk"
subdomain="beta-v2/app"
tokenurl=f'https://{domain}/{subdomain}/api/api-token-auth/'
webserviceurl=f'https://{domain}/{subdomain}/api/'
ai_api_webserviceurl=f'https://{domain}/{subdomain}/api/ai-query/'
credentials={"username":"ai-user","password":"SYSCORE-demo#1234"}

In [8]:
response=get_token_omdAPI(tokenurl,credentials)
token=response.json()['token']

# Model Data Manipulation

In [ ]:
apimethod = 'get_all_models'
apidata = {}
response = call_omdAPI(webserviceurl, token, apimethod, apidata)
print(response.status_code)
model_records=json.loads(response.json()['response']['data'])['records']
df=pd.DataFrame(model_records)
df

##LlamaIndex

In [10]:
import llama_index

In [11]:
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core.graph_stores.simple import SimpleGraphStore
from llama_index.core import Settings

In [34]:
# ✅ Set STORAGE_DIR to the base project directory
STORAGE_DIR = "/content/storage/models"
VECTOR_STORE_DIR = "/content/storage/models"

def initialize_index(llm, embed_model):
    """Initialize the vector index and store it persistently."""
    pathlib.Path(STORAGE_DIR).mkdir(parents=True, exist_ok=True)
    pathlib.Path(VECTOR_STORE_DIR).mkdir(parents=True, exist_ok=True)

    fake_docstore_doc = {
        "text": "This is a placeholder document.",
        "metadata": {"data_id": "placeholder-id"},
    }
    fake_docstore = {"doc_id_1": fake_docstore_doc}

    # ✅ Write docstore and index store
    with open(os.path.join(STORAGE_DIR, "docstore.json"), "w") as f:
        json.dump(fake_docstore, f)

    with open(os.path.join(STORAGE_DIR, "index_store.json"), "w") as f:
        json.dump(fake_docstore, f)

    documents = [Document(text=json.dumps(fake_docstore_doc), metadata=fake_docstore_doc)]

    vector_store = SimpleVectorStore(persist_dir=VECTOR_STORE_DIR)
    StorageContext._default_persist_dir = STORAGE_DIR
    # ✅ Create storage context
    storage_context = StorageContext.from_defaults(
        persist_dir=STORAGE_DIR,
        vector_store=vector_store
    )

    # ✅ Create and persist vector index
    index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, llm=llm, embed_model=embed_model)
    index.storage_context.persist(persist_dir=STORAGE_DIR)

    return index




def add_records_to_index(llm, embed_model, records):
    """Add multiple records to the index."""
    storage_context = StorageContext.from_defaults(persist_dir=STORAGE_DIR)
    index = load_index_from_storage(storage_context, llm=llm, embed_model=embed_model)

    for record in records:
        text = json.dumps(record)
        metadata = record
        new_document = Document(text=text, metadata=metadata)
        index.insert(new_document)  # Insert document into index

    index.storage_context.persist(persist_dir=STORAGE_DIR)
    return index


def add_json_to_index(llm, embed_model, text, metadata):
    """Add a single JSON document to the index."""
    storage_context = StorageContext.from_defaults(persist_dir=STORAGE_DIR)
    index = load_index_from_storage(storage_context, llm=llm, embed_model=embed_model)

    new_document = Document(text=text, metadata=metadata)
    index.insert(new_document)
    index.storage_context.persist(persist_dir=STORAGE_DIR)
    return index

from llama_index.core.response_synthesizers import get_response_synthesizer

def query_index(query, llm, embed_model, retriever_kwargs={"top_k": 1}):
    """Query the stored index."""
    storage_context = StorageContext.from_defaults(persist_dir=STORAGE_DIR)

    # ✅ Try to load index safely
    try:
        index = load_index_from_storage(storage_context, llm=llm, embed_model=embed_model)
    except Exception as e:
        return {"error": f"Failed to load index: {str(e)}"}

    # ✅ Create a query engine
    query_engine = index.as_query_engine(llm=llm, retriever_kwargs=retriever_kwargs)

    # ✅ Run query
    metadata={}
    response = query_engine.query(query)

    if hasattr(response, "metadata") and isinstance(response.metadata, dict):
        first_key = next(iter(response.metadata), None)  # Get the first key
        if first_key:
            metadata=response.metadata[first_key]


    return {"meta_data":metadata}

##Create a model index

In [35]:
index=initialize_index(openai_llm, embed_model)

In [36]:
add_records_to_index(openai_llm, embed_model, model_records)

In [37]:
message="RegionSlotAllocation wise for week 24 to 38 for 2025" #"A Pie chart of region wise slots allocated"

In [38]:
query_response = query_index(message, openai_llm, embed_model)

In [39]:
query_response

{'meta_data': {'app_label': 'modelsApp',
  'model_name': 'MaxSlotAllocation',
  'fields': ['year', 'week', 'product_type', 'slots'],
  'description': 'Model MaxSlotAllocation fields are year, week, product_type, slots'}}

In [ ]:
query_response

In [ ]:
response.text